<a href="https://colab.research.google.com/github/NITHIN-KANDI/PR_PROJECT/blob/main/hmm_gussianemmission_regularzation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.mixture import GaussianMixture
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import ngrams
import nltk

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

class EnhancedHMM:
    def __init__(self, n_states, n_observations, n_mixtures=2, regularization=1e-6):
        self.n_states = n_states
        self.n_observations = n_observations
        self.n_mixtures = n_mixtures
        self.regularization = regularization

        # Initialize transition probabilities (add regularization)
        self.transition_probs = np.full((n_states, n_states), 1.0 / n_states) + regularization

        # Initialize emission probabilities as Gaussian Mixtures
        self.gmms = [GaussianMixture(n_components=n_mixtures, covariance_type='full') for _ in range(n_states)]

        # Initialize start probabilities with regularization
        self.start_probs = np.full(n_states, 1.0 / n_states) + regularization

    def train(self, sequences, state_sequences):
        # Update start probabilities with regularization
        for state_seq in state_sequences:
            self.start_probs[state_seq[0]] += 1
        self.start_probs = (self.start_probs + self.regularization) / (self.start_probs.sum())

        # Update transition probabilities with regularization
        for state_seq in state_sequences:
            for i in range(len(state_seq) - 1):
                self.transition_probs[state_seq[i], state_seq[i + 1]] += 1
        self.transition_probs = (self.transition_probs + self.regularization)
        self.transition_probs /= self.transition_probs.sum(axis=1, keepdims=True)

        # Train Gaussian Mixture Models for emission probabilities
        for state in range(self.n_states):
            state_observations = [obs for seq, state_seq in zip(sequences, state_sequences)
                                  for obs, s in zip(seq, state_seq) if s == state]
            if state_observations:
                self.gmms[state].fit(state_observations)

    def viterbi(self, sequence):
        T = len(sequence)
        if T == 0:
            return [0]  # Return default state if sequence is empty

        # Initialize Viterbi matrices
        viterbi_probs = np.zeros((self.n_states, T))
        backpointer = np.zeros((self.n_states, T), dtype=int)

        # Initialize the first column with Gaussian mixture probabilities
        for s in range(self.n_states):
            viterbi_probs[s, 0] = self.start_probs[s] * np.exp(self.gmms[s].score([sequence[0]]))

        # Fill in the Viterbi matrix
        for t in range(1, T):
            for s in range(self.n_states):
                emission_prob = np.exp(self.gmms[s].score([sequence[t]]))
                transition_emission_probs = viterbi_probs[:, t - 1] * self.transition_probs[:, s] * emission_prob
                viterbi_probs[s, t] = np.max(transition_emission_probs)
                backpointer[s, t] = np.argmax(transition_emission_probs)

        # Backtrace to get the most probable state sequence
        best_path = np.zeros(T, dtype=int)
        best_path[-1] = np.argmax(viterbi_probs[:, T - 1])
        for t in range(T - 2, -1, -1):
            best_path[t] = backpointer[best_path[t + 1], t + 1]

        return best_path


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Load filtered datasets
train_data = pd.read_csv('/content/filtered_train_data.csv')
test_data = pd.read_csv('/content/filtered_test_data.csv')

# Define state mapping and vocabulary
state_mapping = {'truth': 0, 'partial truth': 1, 'partial lie': 2, 'lie': 3}
n_states = len(state_mapping)

# Initialize stopwords and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text_with_bigrams(text):
    # Tokenize, remove stopwords, and stem
    tokens = [stemmer.stem(word) for word in word_tokenize(text.lower()) if word.isalnum() and word not in stop_words]
    bigrams = [' '.join(gram) for gram in ngrams(tokens, 2)]  # Generate bigrams
    return bigrams

# Apply preprocessing and create vocabulary
train_data['tokens'] = train_data['statement'].apply(preprocess_text_with_bigrams)
all_bigrams = [bigram for bigrams in train_data['tokens'] for bigram in bigrams]
vocab = list(set(all_bigrams))
word_to_index = {word: i for i, word in enumerate(vocab)}
n_observations = len(vocab)

# Encode tokens and labels
def encode_tokens(tokens):
    return [[word_to_index.get(token, -1)] for token in tokens if token in word_to_index]

encoded_train_sequences = [encode_tokens(tokens) for tokens in train_data['tokens']]
encoded_state_sequences = [[state_mapping[label]] for label in train_data['label']]


In [4]:
# Initialize and train the enhanced HMM
hmm_model = EnhancedHMM(n_states=n_states, n_observations=n_observations, n_mixtures=2, regularization=1e-6)
hmm_model.train(encoded_train_sequences, encoded_state_sequences)
print("Enhanced HMM model training completed.")


Enhanced HMM model training completed.


In [5]:
def classify_sentence(sentence, hmm_model, word_to_index, state_mapping):
    tokens = preprocess_text_with_bigrams(sentence)
    encoded_tokens = encode_tokens(tokens)
    if not encoded_tokens:
        return "Unknown"  # If no tokens are recognized

    # Use the Viterbi algorithm to find the best state path
    best_path = hmm_model.viterbi(encoded_tokens)
    return list(state_mapping.keys())[best_path[0]]

# Test with a sample sentence
sample_sentence = "This is an entirely true statement."
classification = classify_sentence(sample_sentence, hmm_model, word_to_index, state_mapping)
print(f"The sentence classification is: {classification}")


The sentence classification is: Unknown


In [6]:
# Preprocess the test set with bigrams
test_data['tokens'] = test_data['statement'].apply(preprocess_text_with_bigrams)
encoded_test_sequences = [encode_tokens(tokens) for tokens in test_data['tokens']]
true_labels = [state_mapping[label] for label in test_data['label']]

# Predict labels using the model
predicted_labels = [hmm_model.viterbi(seq)[0] for seq in encoded_test_sequences]

# Evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 0.36
Precision: 0.26
Recall: 0.36
F1 Score: 0.29


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
